## Download data from IDC

Download the DICOM images from IDC


Install utils to process dicom images, process the LIDC dataset, process the images and save them to files, and to download files from GCP stores

In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.0 MB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 62.0 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 52.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 49.1 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 6.3 MB/s  eta 0:00:01
     |████████████████████████████████| 4.0 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████|

Install the utils to sort the DICOM images

In [3]:
!git clone https://github.com/pieper/dicomsort.git

fatal: destination path 'dicomsort' already exists and is not an empty directory.


#### Set up connection to GCP, Gcloud, Big table 
Install utils to query the Bigtables

In [3]:
!pip install google-cloud-bigquery
!pip install --upgrade google-auth-oauthlib
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'
!pip list | grep google-cloud-bigquery

Requirement already up-to-date: google-auth-oauthlib in /opt/conda/lib/python3.8/site-packages (0.4.6)
Requirement already up-to-date: google-cloud-bigquery[bqstorage,pandas] in /opt/conda/lib/python3.8/site-packages (2.30.1)
     |████████████████████████████████| 171 kB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 25.6 MB 63.5 MB/s eta 0:00:01
     |████████████████████████████████| 517 kB 76.1 MB/s eta 0:00:01
google-cloud-bigquery            2.30.1
google-cloud-bigquery-storage    2.10.1


- Load the magic to query bigtables from notebook
- Might need one restart after the previous step for the magic extension to load correctly

In [11]:
%load_ext google.cloud.bigquery

## Pre-requisite to using open CV

- apt-get update
- apt-get install ffmpeg libsm6 libxext6  -y

In [1]:
import pylidc as pl
from pylidc.utils import consensus
import pydicom as dicom
from skimage.measure import find_contours
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import contextlib
import matplotlib.patches as patches
import os
import imageio
import sys
from glob import glob
import os
import cv2
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
import os
from numpy import random
import time
import pandas as pd
from PIL import Image
import contextlib

Check if the pylidc library is able to process the records

In [2]:
pl.query(pl.Scan).count()
pid = 'LIDC-IDRI-0069'#'LIDC-IDRI-0001'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()
scan.study_instance_uid

'1.3.6.1.4.1.14519.5.2.1.6279.6001.303241414168367763244410429787'

Set up connection to the google big table

In [3]:
from google_auth_oauthlib import flow

# TODO: Uncomment the line below to set the `launch_browser` variable.
launch_browser = False
#
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    "client_secret_desktop.json", scopes=["https://www.googleapis.com/auth/bigquery"]
)

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=286006984742-oetffmukptpafo18coob3vgc4f42oppi.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=8ffBFhcGrKXm6dN8pL7Sw9xhXWs4s2&prompt=consent&access_type=offline


Enter the authorization code:  4/1AX4XfWi_C1CJwEY7qWpKpMNPu7cQ6X0lHiBz7edkdGG1Yfxxmt0nAXLIO4A


In [8]:
from google.cloud.bigquery import magics
magics.context.credentials = credentials

Note: mandatory next step

> Configure gsutil in a console
> - gsutil config

In [9]:
myProjectID = "positive-sector-330514"
#myProjectID = "idc-external-012"

In [12]:
%%bigquery CT_series --project=$myProjectID 

WITH
  all_lidc_ct_series AS (
  SELECT
    DISTINCT(SeriesInstanceUID),
    StudyInstanceUID,  
    PatientID,
    SliceThickness,
    ARRAY_TO_STRING(PixelSpacing,"/") as pixelspa 
  FROM
    `canceridc-data.idc_views.dicom_all`
  WHERE
    Modality = "CT"
    AND collection_id = "lidc_idri")
SELECT PatientID,SliceThickness,pixelspa,StudyInstanceUID,SeriesInstanceUID FROM
  all_lidc_ct_series
ORDER BY
  PatientID

Downloading: 100%|██████████| 1018/1018 [00:02<00:00, 439.44rows/s]


In [13]:
total_patients = len(np.unique(CT_series['PatientID']))
chosen_patients = len(np.unique(CT_series[(CT_series["SliceThickness"].astype(float)>=1.0)]['PatientID']))
print("Choosing patients with slice thickness > 1.0")
print("Total patients is {} and chosen patients are {}".format(total_patients, chosen_patients))
ct_series_HR = CT_series[(CT_series["SliceThickness"].astype(float)>=1.0)]
print("Shape of the raw dataset is {}".format(ct_series_HR.shape))

to_download = []
base_gs_uri = 'gs://idc-tcia-lidc-idri/dicom/'

for i in range(ct_series_HR.shape[0]):
    to_download.append(base_gs_uri + ct_series_HR.iloc[i]['StudyInstanceUID'] + '/' + ct_series_HR.iloc[i]['SeriesInstanceUID'])

manifest_df = pd.DataFrame({"gcs":to_download},columns=["gcs"])

manifest_df.to_csv("gcs_paths_all.txt",header=False, index=False)

Choosing patients with slice thickness > 1.0
Total patients is 1010 and chosen patients are 971
Shape of the raw dataset is (979, 5)


In [14]:
!head gcs_paths_all.txt

gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.298806137288633453246975630178/1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636438705059720603192
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.490157381160200744295382098329/1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417051974713149104919
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276577556143013894866/1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615202213033480003264
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.191425307197546732281885591780/1.3.6.1.4.1.14519.5.2.1.6279.6001.323541312620128092852212458228
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.190188259083742759886805142125/1.3.6.1.4.1.14519.5.2.1.6279.6001.129007566048223160327836686225
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.324680252006411183918098592500/1.3.6.1.4.1.14519.5.2.1.6279.6001.132817748896065918417924920957
gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.

#### Download the DICOM images

- Define the paths where we will download the DICOM images from GCP and then sort them.
- Create the train / val / test split based on the manifest URLs

In [16]:
idxs = np.arange(manifest_df.shape[0])
np.random.seed(2011)
np.random.shuffle(idxs)
split_ratio = 0.05
data_idxs = idxs[int(manifest_df.shape[0]*split_ratio):]
test_idxs = idxs[:int(manifest_df.shape[0]*split_ratio)]
split_ratio_val = 0.2
train_idxs = data_idxs[int(data_idxs.shape[0]*split_ratio_val):]
val_idxs = data_idxs[:int(data_idxs.shape[0]*split_ratio_val)]
print("Splits train:{}, val: {}, test: {}".format(train_idxs.shape[0],val_idxs.shape[0],test_idxs.shape[0]))

Splits train:745, val: 186, test: 48


#### Routine to download and sort the DICOM images

You can adjust how many you wish to download at a time, to conserve space on the disk

In [23]:
import os, shutil
import glob

def download_data(idxs, data_dir, dtype='train', paging=True):
    
    download_dir = data_dir + "/" + dtype + "/" + 'gcsfiles/'
    sorted_dir = data_dir + "/" + dtype + "/" + 'sorted_gcsfiles/'
    if not os.path.exists(download_dir):
        print("Creating directory {}".format(download_dir))
        os.makedirs(download_dir)
    else:
        print("Directory {} exists".format(download_dir))

    if not os.path.exists(sorted_dir):
        print("Creating directory {}".format(sorted_dir))
        os.makedirs(sorted_dir)
    else:
        print("Directory {} exists".format(sorted_dir))
    
    manifest = manifest_df.iloc[idxs]
    
    if paging:
        for i in tqdm(range(manifest.shape[0])):
            url = manifest.iloc[i].gcs

            #download the DICOM images
            download_command = "gsutil -u {} -m cp -r {} {} >/dev/null 2>&1".format(myProjectID, url, download_dir)
            os.system(download_command)

            if (i > 1) and (i % 50 == 0): # process 50 patients at a time, to save space on disk
                # sort the images
                sort_command = "python dicomsort/dicomsort.py -u {} {}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm".format(download_dir, sorted_dir)
                os.system(sort_command)

                # clean up the download dir
                files = glob.glob(download_dir + "/*")
                for j in tqdm(range(len(files))):
                    if os.path.isdir(files[j]):
                        shutil.rmtree(files[j])
    else:
        #download all
        for i in tqdm(range(manifest.shape[0])):
            url = manifest.iloc[i].gcs
            #download the DICOM images
            download_command = "gsutil -u {} -m cp -r {} {} >/dev/null 2>&1".format(myProjectID, url, download_dir)
            os.system(download_command)
            
        #sort all
        sort_command = "python dicomsort/dicomsort.py -u {} {}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm".format(download_dir, sorted_dir)
        os.system(sort_command)
        
        #clean up the download dir
        files = glob.glob(download_dir + "/*")
        for j in tqdm(range(len(files))):
            if os.path.isdir(files[j]):
                shutil.rmtree(files[j])

#### Configure the data directories

In [24]:
data_dir = '/app/data/lidc/dicom/'

In [25]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
%%time
download_data(train_idxs, data_dir, 'train', True)

  0%|          | 0/745 [00:00<?, ?it/s]

Creating directory /app/data/lidc/dicom//train/gcsfiles/
Creating directory /app/data/lidc/dicom//train/sorted_gcsfiles/


  8%|▊         | 56/745 [09:45<2:01:17, 10.56s/it]

In [26]:
%%time
download_data(val_idxs, data_dir, 'val', True)

  0%|          | 0/186 [00:00<?, ?it/s]

Creating directory /app/data/lidc/dicom//val/gcsfiles/
Creating directory /app/data/lidc/dicom//val/sorted_gcsfiles/


100%|██████████| 186/186 [26:47<00:00,  8.64s/it]

CPU times: user 578 ms, sys: 3.33 s, total: 3.91 s
Wall time: 26min 47s


In [27]:
%%time
download_data(test_idxs, data_dir, 'test', False)

  0%|          | 0/48 [00:00<?, ?it/s]

Creating directory /app/data/lidc/dicom//test/gcsfiles/
Creating directory /app/data/lidc/dicom//test/sorted_gcsfiles/


100%|██████████| 48/48 [00:01<00:00, 47.28it/s]

CPU times: user 165 ms, sys: 994 ms, total: 1.16 s
Wall time: 4min 38s


In [19]:
def set_dicom_path(dicom_path):
    f = open ('/root/.pylidcrc','w')
    f.write('[dicom]'+'\n')
    f.write('path =' + dicom_path +'\n')
    f.write('warn = True')
    f.close()

In [20]:
def load_vol_for_patient(patient_id, dicom_path):
    set_dicom_path(dicom_path)
    #patient_id = [patient_id
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_([patient_id]))[0]
    nodules_annotation = scan.cluster_annotations()
    with contextlib.redirect_stdout(None):
        vol = scan.to_volume()
    return vol, nodules_annotation

In [27]:
dicom_root_dir = '/app/data/lidc/dicom/train/sorted_gcsfiles/'

In [28]:
# test
pat_id = 'LIDC-IDRI-0040'
vol, nodules_annotation = load_vol_for_patient(pat_id, dicom_root_dir)
print("There are {} nodules annotated for the patient {}".format(len(nodules_annotation), pat_id))

There are 4 nodules annotated for the patient LIDC-IDRI-0040
